In [14]:
import matplotlib.pyplot as plt
import IPython.display as Ipd
import pandas as pd
import torch
from pathlib import Path
import numpy as np

from b2aiprep.process import SpeechToText
from b2aiprep.process import Audio, specgram

from b2aiprep.dataset import VBAIDataset


# load data
GeMAPS_demo_df = pd.read_csv("GeMAPS_demo_df2.csv")

path = '/home/bridge2ai/Desktop/bridge2ai-data/bids_with_sensitive_recordings/'
dataset = VBAIDataset(path)

# load and clean participant and demographics dataframes
participant_df = dataset.load_and_pivot_questionnaire('participant')
demographics_df = dataset.load_and_pivot_questionnaire('qgenericdemographicsschema')
demographics_df_clean = demographics_df[demographics_df.duplicated('record_id', keep=False) == False]
participant_df_clean = participant_df[participant_df.duplicated('record_id', keep=False) == False]

# merge cleaned participant and demographic dataframes
# Perform an inner join on 'record_id'
participant_demo_df = pd.merge(
    demographics_df_clean, 
    participant_df_clean, 
    on='record_id', 
    how='inner'
)


GeMAPS_demo_df = pd.merge(
    GeMAPS_demo_df,
    participant_demo_df[["record_id", "gender_identity"]],
    on="record_id",
    how="inner"
)

In [15]:
# Import tableone
try:
    from tableone import TableOne, load_dataset
except (ModuleNotFoundError, ImportError):
    # install on Colab
    !pip install tableone
    from tableone import TableOne, load_dataset


table1 = TableOne(
    GeMAPS_demo_df,
    groupby='depression',
    continuous=['age'],
    categorical=['gender_identity', 'osa','laryng_cancer','benign_cord_lesion',
                 'rrp','spas_dys','voc_fold_paralysis','alz_dementia_mci','als','parkinsons',
                 'alcohol_subst_abuse','add_adhd','asd','bipolar','bpd','ed','insomnia','ocd',
                 'asthma','airway_stenosis','chronic_cough','copd'],
    dip_test=True, 
    normal_test=True, 
    tukey_test=True,
    pval=True,
    smd=True
)

table1

Grouped by depression                                                             
                                                                                   Missing      Overall          0.0          1.0 SMD (0.0,1.0) P-Value
n                                                                                                   105           77           28                      
age, mean (SD)                                                                           0  54.1 (18.7)  59.6 (14.9)  38.9 (20.0)        -1.169  <0.001
osa, n (%)                 0.0                                                               101 (96.2)    75 (97.4)    26 (92.9)         0.212   0.288
                           1.0                                                                  4 (3.8)      2 (2.6)      2 (7.1)                      
add_adhd, n (%)            0.0                                                                96 (91.4)    75 (97.4)    21 (75.0)         0.687   0.001
                           1.0                                                                  9 (8.6)      2 (2.6)     7 (25.0)                      
asd, n (%)                 0.0                                                              105 (100.0)   77 (100.0)   28 (100.0)        <0.001   1.000
bipolar, n (%)             0.0                                                               103 (98.1)    76 (98.7)    27 (96.4)         0.148   0.464
                           1.0                                                                  2 (1.9)      1 (1.3)      1 (3.6)                      
bpd, n (%)                 0.0                                                               103 (98.1)   77 (100.0)    26 (92.9)           nan   0.069
                           1.0                                                                  2 (1.9)                   2 (7.1)                      
ed, n (%)                  0.0                                                               104 (99.0)   77 (100.0)    27 (96.4)           nan   0.267
                           1.0                                                                  1 (1.0)                   1 (3.6)                      
insomnia, n (%)            0.0                                                               101 (96.2)    76 (98.7)    25 (89.3)         0.404   0.057
                           1.0                                                                  4 (3.8)      1 (1.3)     3 (10.7)                      
ocd, n (%)                 0.0                                                               103 (98.1)   77 (100.0)    26 (92.9)           nan   0.069
                           1.0                                                                  2 (1.9)                   2 (7.1)                      
asthma, n (%)              0.0                                                                99 (94.3)    73 (94.8)    26 (92.9)         0.081   0.656
                           1.0                                                                  6 (5.7)      4 (5.2)      2 (7.1)                      
airway_stenosis, n (%)     0.0                                                                57 (54.3)    32 (41.6)    25 (89.3)         1.160  <0.001
                           1.0                                                                48 (45.7)    45 (58.4)     3 (10.7)                      
chronic_cough, n (%)       0.0                                                               102 (97.1)    74 (96.1)   28 (100.0)           nan   0.563
                           1.0                                                                  3 (2.9)      3 (3.9)                                   
copd, n (%)                0.0                                                               104 (99.0)    76 (98.7)   28 (100.0)           nan   1.000
                           1.0                                                                  1 (1.0)      1 (1.3) 